In [1]:
import os
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.download.n

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Reviews").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [6]:
review_df = df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982| 2015-08-31|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953| 2015-08-31|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933| 2015-08-31|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886| 2015-08-31|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381| 2015-08-31|
|R1DGA6UQIVLKZ7|    2781942|B007B5V092|     152891509| 2015-08-31|
| RLQT3V8SMNIBH|     707292|B00IODHGVG|     717335277| 2015-08

In [15]:
products_df = df.select(['product_id','product_title']).dropDuplicates(['product_id','product_title'])
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
|B009W857SG|TETC Cable (3 Fee...|
|B007LG453Y|Tmvel TVM-UK2 Uni...|
|B00KFSO0SA|Monoprice 11149 3...|
|B00IH5U050|Digital Army Camo...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B00YDMW0DQ|Sound Intone HD30...|
|B00AR0VCXW|Pro-Ject Speed Bo...|
|B006BE6WK6|12 PIECE Ear Hook...|
|B00TM1CT98|Bose Acoustimass ...|
|B00P8D911I|AuraBeam Samsung ...|
+----------+--------------------+
only showing top 20 rows



In [18]:
customers_df = df.select(['customer_id']).dropDuplicates(['customer_id'])
customers_df.show()


+-----------+
|customer_id|
+-----------+
|   11608395|
|   22848554|
|   43490052|
|   16281356|
|    3223564|
|   44662810|
|   41681532|
|   16336729|
|   31425737|
|   31693763|
|   12536186|
|    7328230|
|   52857540|
|   40549446|
|     485535|
|    1350260|
|   41257827|
|   12374662|
|   45193257|
|     144481|
+-----------+
only showing top 20 rows



In [19]:
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config) 
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config) 
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config) 